In [567]:
import pandas as pd
import ast
import numpy as np

In [568]:
path = 'contrastive.csv'
df = pd.read_csv(path, index_col=0)[['premise', 'hypothesis', 'prompt', 'rationale']]

In [569]:
df['hypothesis'] = df['hypothesis'].apply(ast.literal_eval)

In [570]:
df.rationale = df.rationale.astype(str)
df['split'] = df.rationale.apply(lambda x: x.lower().split('most likely answer'))
df.split = df.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [571]:
df.head(1)

,premise,hypothesis,prompt,rationale,split
0,"""There are 10 apples on an apple tree. Three ...","[park, coloring book, garden center, math prob...","Questions: ""There are 10 apples on an apple tr...",1. The most likely answer is **(D) math proble...,is **(d) math problem**. this is an example o...


In [572]:
# get correct index
df['correct_index'] = [{} for _ in range(len(df))]
for index, row in df.iterrows():
    for choice in row['hypothesis']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice.lower())

In [573]:
df['LLM_answer'] = df.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [574]:
df[df.LLM_answer.isna()]

,premise,hypothesis,prompt,rationale,split,correct_index,LLM_answer


In [575]:
# read paper's answer
paper = pd.read_csv('paper.csv', index_col=0)

In [576]:
paper['premise'] = paper.input.str.split('\n').apply(lambda x: x[0])
paper.set_index('premise', inplace=True)
paper.index = paper.index.str.strip()

In [577]:
df.set_index('premise', inplace=True)

In [578]:
paper['my_label'] = df['LLM_answer']

In [579]:
sum(paper.label == paper.my_label) / len(paper)

0.8229134585771481

In [580]:
df.to_csv('contrastive - full.csv')

In [499]:
df

,hypothesis,prompt,rationale,split,correct_index,LLM_answer
premise,,,,,,
"""There are 10 apples on an apple tree. Three fall off. Now there are X apples."" What is this an example of?","[park, coloring book, garden center, math prob...","Questions: ""There are 10 apples on an apple tr...",1. The correct answer is (D) math problem. Thi...,1. the correct answer is (d) math problem. thi...,{'math problem': 29},math problem
"A John is a bum. Much like the stereotype, he lives near this sort of transportation infrastructure. Where does he live?","[bus depot, beach, train station, bridge, bridge]",Questions: A John is a bum. Much like the ste...,2. The correct answer is (D) bridge. A John is...,2. the correct answer is (d) bridge. a john is...,{'bridge': 29},bridge
"A bad person places little value on being honest, acting without pretense or being what?","[excellent, upright, premium, competent, sincere]",Questions: A bad person places little value on...,3. The correct answer is (E) sincere. A bad pe...,3. the correct answer is (e) sincere. a bad pe...,{'sincere': 29},sincere
"A bald eagle flies over St. Paul, where is it?","[texas, thermal, minnesota, canada, photograph]","Questions: A bald eagle flies over St. Paul, w...",4. The correct answer is (C) minnesota. A bald...,4. the correct answer is (c) minnesota. a bald...,{'minnesota': 29},minnesota
"A battleship is a powerful vessel. If you need something similar but faster, what would you use?","[yatch, corvette, aircraft carrier, destroyer,...",Questions: A battleship is a powerful vessel. ...,5. The correct answer is (D) destroyer. A batt...,5. the correct answer is (d) destroyer. a batt...,{'destroyer': 29},destroyer
...,...,...,...,...,...,...
Where is a pleasure garden likely to have orginated?,"[fairy tale, resort town, english courtyard, p...",NaN,2. The correct answer is (D) palace. A pleasur...,2. the correct answer is (d) palace. a pleasur...,"{'fairy tale': 229, 'resort town': 243, 'engli...",palace
Where is a snake likely to startle a farmer?,"[tropical forest, tree, pet shops, book store,...",NaN,3. The correct answer is (E) field. A snake is...,3. the correct answer is (e) field. a snake is...,"{'tropical forest': 127, 'tree': 146, 'book st...",tropical forest
Where is an apple tree likely found in abundance?,"[south pole, park, vineyard, farmland, orchid]",NaN,4. The correct answer is (E) orchard. An apple...,4. the correct answer is (e) orchard. an apple...,"{'south pole': 140, 'park': 154, 'vineyard': 1...",south pole
